 # Projet web Mining : 
 # FEUDJIO NGOUANET VALDES VOTEL
 # LAGNOL    

### PARTIE 1 : WEB SCRAPING DE LA BASE DE DONNEES

#### installation des packages de base

In [1]:
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/francetvinfo.json
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/tvinfo-sources.json
!pip install feedparser
!pip install newspaper3k
!python -m spacy download fr_core_news_md

--2024-12-06 14:15:14--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/francetvinfo.json
Resolving people.irisa.fr (people.irisa.fr)... 131.254.254.107
Connecting to people.irisa.fr (people.irisa.fr)|131.254.254.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5887584 (5.6M) [application/json]
Saving to: 'francetvinfo.json.1'

     0K .......... .......... .......... .......... ..........  0% 6.36M 1s
    50K .......... .......... .......... .......... ..........  1% 2.40M 2s
   100K .......... .......... .......... .......... ..........  2% 1.26M 2s
   150K .......... .......... .......... .......... ..........  3% 1.83M 3s
   200K .......... .......... .......... .......... ..........  4% 1.30M 3s
   250K .......... .......... .......... .......... ..........  5% 1.07M 3s
   300K .......... .......... .......... .......... ..........  6% 1.18M 3s
   350K .......... .......... .......... .......... ..........  6% 1.09M 4s
   400K ..........

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/81.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/81.3 kB ? eta -:--:--
   ---------- ----------------------------- 20.5/81.3 kB 330.3 kB/s eta 0:00:01
   -------------------- ------------------- 41.0/81.3 kB 393.8 kB/s eta 0:00:01
   ---------------------------------------- 81.3/81.3 kB 652.3 kB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6061 sha256=cbea12c2fee0ee22d012a8e2fecd3f5fd8b0d83350948753f08c611ec4625b65
  Stored in directory: c:\users\lenovo pc\appdata\local\pip\cache\wheels\3b\25\2a\105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/7.4 MB ? eta -:--:--
     -----------------------

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "c:\Users\LENOVO PC\anaconda3\Lib\site-packages\spacy\__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "c:\Users\LENOVO PC\anaconda3\Lib\site-packages\spacy\errors.py", line 3, in <module>
    from .compat import Literal
  File "c:\Users\LENOVO PC\anaconda3\Lib\site-packages\spacy\compat.py", line 39, in <module>
    from thinc.api import Optimizer  # noqa: F401
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LENOVO PC\anaconda3\Lib\site-packages\thinc\api.py", line 23, in <module>
    from .layers import (
  File "c:\Users\LENOVO PC\anaconda3\Lib\site-packages\thinc\layers\__init__.py", line 12, in <module>
    from .concatenate import concatenate
  File "c:\Users\LENOVO PC\anaconda3\Lib\site-packages\thinc\layers\concatenate.py"

In [74]:
import json

with open('tvinfo-sources.json','r') as f1:
    tvinfo=json.load(f1)
    
with open('francetvinfo.json.','r') as f2:
    francetvinfo=json.load(f2)

#### Chargement des données deja existantes et des données sur les flus RSS

In [99]:
import json
import ssl
import feedparser as fp

# (a) Se prémunir contre le blocage de commande
ssl._create_default_https_context = ssl._create_unverified_context

# (b) Lire la base de données existante (francetvinfo.json)
with open("francetvinfo.json", "r", encoding="utf-8") as f:
    existing_data = json.load(f)


# (c) Lire les flux RSS listés dans tvinfo-sources.json
with open("tvinfo-sources.json", "r", encoding="utf-8") as f:
    rss_sources = json.load(f)


#### Selection des données qui ne sont pas encore dans la base de données existante

In [100]:
from newspaper import Article

# Extraire les liens existants pour éviter les doublons
existing_links = existing_data.keys()

# Scanner les flux RSS et repérer les nouveaux articles
new_articles = {}

for category, url in rss_sources.items():
    print(f"Lecture du flux RSS : {url}")
    data = fp.parse(url)

    # Parcourir les articles du flux RSS
    for item in data.entries:
        if item.link not in existing_links:  # Vérifier si l'article est nouveau
            try:
                # Utiliser newspaper pour extraire les informations détaillées
                article = Article(item.link)
                article.download()
                article.parse()

                # Ajouter les détails de l'article dans new_articles
                new_articles[item.link] = {
                    "title": article.title or item.title,
                    "date": article.publish_date if article.publish_date else (item.published if hasattr(item, "published") else "N/A"),
                    "author": article.authors or item.get("author", "Inconnu"),
                    "category": category,
                    "content": article.text or item.get("description", "Pas de contenu"),
                    "image_link": article.top_image or "Pas d'image"
                }
            except Exception as e:
                print(f"Erreur lors du traitement de l'article : {item.link}\n{e}")

print(f"{len(new_articles)} nouveaux articles ajoutés :")


Lecture du flux RSS : https://www.francetvinfo.fr/france.rss
Lecture du flux RSS : https://www.francetvinfo.fr/monde/europe.rss
Lecture du flux RSS : https://www.francetvinfo.fr/economie/entreprises.rss
60 nouveaux articles ajoutés :


In [105]:
# On affiche les liens des nouveaux articles parus das la base de données
for key in new_articles.keys():
    print(key)

https://www.francetvinfo.fr/marseille/directrice-de-la-prison-des-baumettes-menacee-de-mort-deux-suspects-ecroues-pour-tentative-d-assassinat_6940289.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/culture/patrimoine/incendie-de-notre-dame-de-paris/reouverture/reouverture-de-notre-dame-de-paris-quelles-seront-les-regles-pour-circuler-pendant-la-ceremonie-ce-week-end_6940274.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/sante/hopital/mort-de-lucas-a-l-hopital-d-hyeres-en-2023-un-rapport-pointe-le-retard-du-diagnostic-et-plusieurs-dysfonctionnements-aux-urgences_6939890.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/marseille/menacee-de-mort-la-directrice-de-la-prison-des-baumettes-a-marseille-temporairement-eloignee-de-ses-fonctions_6939776.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/economie/crise/blocus-des-agriculteurs/colere-des-agriculteurs-le-prefet-de-tarn-et-garonne-condamne-l-action-d-une-trentaine-de-manifestants-qui-ont-vise-des-batiments-public

In [108]:
# faire une nouvelle base de données en y ajoutants de nouvelles données

francetvinfo_total=existing_data.update(new_articles)
len(existing_data)

2157